In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType, StringType, ArrayType, DoubleType
from pyspark.sql.functions import udf, col
import pyspark.sql.functions as func
import re

### Initialize Spark session and read CSV file

In [2]:
spark1 = SparkSession.builder.appName('spark').getOrCreate()
df = spark1.read.csv('MetObjects.csv',inferSchema=True,header=True)

### Select required columns

In [3]:
df_1= df.select("Object ID","Dimensions")

### Some EDA to get taste of data

In [136]:
df_1.select('Dimensions').distinct().show(truncate=False)

+----------------------------------------------------+
|Dimensions                                          |
+----------------------------------------------------+
|59 1/4 x 33 x 26 3/8 in. (150.5 x 83.8 x 67 cm)     |
|6 1/4 x 16 1/2 in. (15.9 x 41.9 cm)                 |
|40 1/2 x 28 5/8 x 33 in. (102.9 x 72.7 x 83.8 cm)   |
|15 3/4 x 11 x 11 in. (40 x 27.9 x 27.9 cm)          |
|15 3/4 x 8 3/4 x 5 1/4 in. (40 x 22.2 x 13.3 cm)    |
|4 1/4 x 3 1/4 x 2 in. (10.8 x 8.3 x 5.1 cm)         |
|21 1/4 x 27 1/2 in. (54 x 69.9 cm)                  |
|H. 6 3/4 in. (17.1 cm); Diam. 3 3/4 in. (9.5 cm)    |
|8 5/8 x 5 in. (21.9 cm)                             |
|H. 11 3/16 in. (28.4 cm)                            |
|9 5/8 x 5 1/2 in. (24.4 x 14 cm)                    |
|20 x 13 1/2 in. (50.8 x 34.3 cm)                    |
|30 1/2 x 22 3/4 x 13 1/2 in. (77.5 x 57.8 x 34.3 cm)|
|28 x 38 in. (71.1 x 96.5 cm)                        |
|24 x 14 in. (61 x 35.6 cm)                          |
|38 3/8 x 

In [141]:
df_1.sample(withReplacement=False, fraction=0.01).show(truncate=False)

+---------+----------------------------------------------------+
|Object ID|Dimensions                                          |
+---------+----------------------------------------------------+
|92       |23 1/4 x 23 1/2 in. (59.1 x 59.7 cm)                |
|189      |38 3/4 x 21 1/2 x 18 in. (98.4 x 54.6 x 45.7 cm)    |
|198      |36 x 22 1/2 x 19 1/4 in. (91.4 x 57.2 x 48.9 cm)    |
|216      |43 x 28 1/4 x 30 in. (109.2 x 71.8 x 76.2 cm)       |
|228      |40 1/2 x 26 x 24 3/4 in. (102.9 x 66 x 62.9 cm)     |
|810      |H. 5 1/2 in. (14 cm); Diam. 8 1/4 in. (21 cm)       |
|1005     |Diam. 6 1/4 in. (15.9 cm)                           |
|1055     |3 1/8 x 2 1/4 in. (7.9 x 5.7 cm)                    |
|1091     |H. 4 3/4 in. (12.1 cm); Diam. 9 1/4 in. (23.5 cm)   |
|1151     |5 5/8 x 12 x 6 1/4 in. (14.3 x 30.5 x 15.9 cm)      |
|null     |null                                                |
|1161     |28 1/2 x 17 1/2 x 17 1/2 in. (72.4 x 44.5 x 44.5 cm)|
|1347     |H. 9 3/4 in. (

#### The Dimensions column has a lot of messy data:
* null
* no information about dimensions

#### Dimensions in cm are always within brackets

### Select rows with dimensions in cms

In [166]:
df_2=df_1.filter(df_1.Dimensions.contains('cm'))

### Select the values within brackets

In [167]:
@udf(returnType=StringType())
def dimensions_in_cm(s):
     q=s[s.find("(")+1:s.find(")")]
     return q

In [168]:
df_3 = df_2.withColumn("dimensions_in_cm", dimensions_in_cm(col("Dimensions")))

### Convert dimensions(L or L x B or L x B x H) to array of dimensions. The unit 'cm' is ignored.

In [169]:
@udf(returnType=ArrayType(StringType()))
def only_dimensions_in_cm(s):
     return re.findall(r"[-+]?\d*\.\d+|\d+", s)

In [170]:
df_4 = df_3.withColumn("dimensions_in_cm_array", only_dimensions_in_cm(col("dimensions_in_cm")))

In [171]:
df_4.show(30)

+---------+--------------------+--------------------+----------------------+
|Object ID|          Dimensions|    dimensions_in_cm|dimensions_in_cm_array|
+---------+--------------------+--------------------+----------------------+
|        3|Diam. 11/16 in. (...|              1.7 cm|                 [1.7]|
|        4|Diam. 11/16 in. (...|              1.7 cm|                 [1.7]|
|        5|Diam. 11/16 in. (...|              1.7 cm|                 [1.7]|
|        6|Diam. 11/16 in. (...|              1.7 cm|                 [1.7]|
|        7|Diam. 11/16 in. (...|              1.7 cm|                 [1.7]|
|        8|Diam. 11/16 in. (...|              1.7 cm|                 [1.7]|
|        9|Diam. 11/16 in. (...|              1.7 cm|                 [1.7]|
|       15|Diam. 1/2 in. (1....|              1.3 cm|                 [1.3]|
|       16|Diam. 1 1/8 in. (...|              2.9 cm|                 [2.9]|
|       17|Diam. 1 1/8 in. (...|              2.9 cm|                 [2.9]|

In [13]:
df_4.printSchema()

root
 |-- Object ID: string (nullable = true)
 |-- Dimensions: string (nullable = true)
 |-- dimensions_in_cm: string (nullable = true)
 |-- dimensions_in_cm_array: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [14]:
s=['101.6', '64.8', '87.6']

In [15]:
[int(i) if i.isdigit() else float(i) for i in s]

[101.6, 64.8, 87.6]

### Convert Array of Strict to Array of Float

In [16]:
@udf(returnType=ArrayType(FloatType()))
def only_dimensions_in_cm(s):
     return [float(i) for i in s]

In [17]:
df_5 = df_4.withColumn("dimensions_in_cm_array_float", only_dimensions_in_cm(col("dimensions_in_cm_array")))

In [18]:
df_5.show(30)

+---------+--------------------+--------------------+----------------------+----------------------------+
|Object ID|          Dimensions|    dimensions_in_cm|dimensions_in_cm_array|dimensions_in_cm_array_float|
+---------+--------------------+--------------------+----------------------+----------------------------+
|        3|Diam. 11/16 in. (...|              1.7 cm|                 [1.7]|                       [1.7]|
|        4|Diam. 11/16 in. (...|              1.7 cm|                 [1.7]|                       [1.7]|
|        5|Diam. 11/16 in. (...|              1.7 cm|                 [1.7]|                       [1.7]|
|        6|Diam. 11/16 in. (...|              1.7 cm|                 [1.7]|                       [1.7]|
|        7|Diam. 11/16 in. (...|              1.7 cm|                 [1.7]|                       [1.7]|
|        8|Diam. 11/16 in. (...|              1.7 cm|                 [1.7]|                       [1.7]|
|        9|Diam. 11/16 in. (...|              

### To make sure an art/object fits in a particular space, the maximum dimension of the art should be less than or equal to the space provided 

In [150]:
@udf(returnType=FloatType())
def max_dimensions(s):
     if s:
        return max(s)

In [151]:
df_6 = df_5.withColumn("max_dimension", max_dimensions(col("dimensions_in_cm_array_float")))

In [153]:
df_6.select('Object ID','dimensions_in_cm_array_float','max_dimension').show(40)

+---------+----------------------------+-------------+
|Object ID|dimensions_in_cm_array_float|max_dimension|
+---------+----------------------------+-------------+
|        3|                       [1.7]|          1.7|
|        4|                       [1.7]|          1.7|
|        5|                       [1.7]|          1.7|
|        6|                       [1.7]|          1.7|
|        7|                       [1.7]|          1.7|
|        8|                       [1.7]|          1.7|
|        9|                       [1.7]|          1.7|
|       15|                       [1.3]|          1.3|
|       16|                       [2.9]|          2.9|
|       17|                       [2.9]|          2.9|
|       18|                       [2.9]|          2.9|
|       19|                       [2.9]|          2.9|
|       20|                       [2.9]|          2.9|
|       21|                       [2.9]|          2.9|
|       22|                       [1.3]|          1.3|
|       23

### Make an alias to Object ID to object_id and save it as a parquet file.
### Reasons behind parquet:
* Can be distributed across machines like HDFS. Hence parallel processing is possible.
* Contains schema within it. This makes it optimized for query performance and minimizing I/O.
* Parquet also supports very efficient compression and encoding schemes.

In [156]:
df_7= df_6.select(col("Object ID").alias("object_id"), col("max_dimension"))

In [159]:
df_7.write.parquet("met1.parquet")

### The function does_it_fit takes in the object_id and the maximum dimension (this has been assumed) of the space.
* Returns 'Cant say for sure. Please check the catalog for more details' if the object_id is not present or if the object had no information about the dimension in the first place.
* Returns 'It will fit' if the maximum dimension of the object is less than dimension of the space provided.
* Returns 'It will not fit' if the maximum dimension of the object is not less than dimension of the space provided.

In [160]:
def does_it_fit(object_id, dimensions):
        parDF1=spark1.read.parquet("met1.parquet")
        parDF2 = parDF1.filter((parDF1.object_id == object_id))
        if parDF2.rdd.isEmpty():
            print("Cant say for sure. Please check the catalog for more details")
        else:
            parDF3 = parDF2.filter(parDF2.max_dimension < dimensions)
            if parDF3.rdd.isEmpty():
                print("It will not fit")
            else:
                print("It will fit")

### Some examples of does_it_fit in action

#### Object_id 14 had no information about the dimension

In [165]:
does_it_fit(14,.45)

Cant say for sure. Please check the catalog for more details


#### Object_id 33 had maximum dimension of 8.9

In [179]:
does_it_fit(33,45)

It will fit


#### Object_id 39 had maximum dimension of 27.9

In [182]:
does_it_fit(39,27)

It will not fit
